## Prototype, choix des projets

Principe:
On lit dans le fichier Excel les préférences de chacun.
Un nombre d'utilisateurs "bidons" sont rajoutés pour arriver à 64 personnes
On construit une matrice de coût pour chaque personne et chaque place dans un projet.

Choix de coût arbitraire ici:
- 0 pour choix 1
- 10 pour choix 2
- 20 pour choix 3
- 40 pour choix 4
- 60 pour choix 5
- 200 si hors classement

**On pourrait à la place demander de noter chaque projets ("très intéressé", "intéressé", "peu intéressé", "je ne veux pas le faire", "je n'ai pas le droit") avec un coût associé.** Cela reflètera sans doute mieux les préférences de chacun pour la répartition initiale.

On distingue les places de chaque projet:
- place n°1 : favorise les math
- place n°2 : favorise l'info
- place n°3 : favorise la gestion de projets
- place n°4 : sans préférence

La place n°4 est aussi la seule qui accepte un utilisateur bidon (pour éviter d'avoir des projets à moins de 3 personnes)

Ici, une faible pondération a été utilisée qui ne prend pas le pas sur les préférences. Il suffit de la mettre à zéro pour ne pas tenir compte du tout de ce critère. Dans le cas du prototype, j'ai l'impression que ça influe quand même beaucoup (ça enlève une part de hasard pour savoir qui va sur les projets qui intéressent beaucoup ou au contraire n'intéressent personne).

Un peu de bruit aléatoire est finalement ajouté à la matrice de coût pour briser les symétries (si deux personnes font exactement le même choix et qu'il y n'y a qu'une place, c'est de là que viendra la décision en dernier ressort)

La matrice est passé dans une fonction d'optimisation qui affecte à chaque personne un projet de sorte à minimiser le coût total (cf: problème d'assignation, algorithme hongrois).

Il y a encore beaucoup d'aléa, en refaisant plusieurs foid le tirage j'ai terminé presque à chaque fois dans un groupe différent.


In [1]:
import numpy as np
import pandas as pd
# La fonction qui permet de résoudre le problème
from scipy.optimize import linear_sum_assignment

### Import des données

In [2]:
df_choix= pd.read_excel('~/MS-IA/FilRouge/Fil Rouge (réponses).xlsx')

In [3]:
#Delete duplicate submissions, keep only last
df_choix = df_choix.drop_duplicates(subset='Email',keep='last')
print(f"{len(df_choix)} participants pour le moment")

53 participants pour le moment


In [4]:
df_choix

,Horodateur,Email,Choix 1,Choix 2,Choix 3,Choix 4,Choix 5,Maths,Informatique,Gestion de projet,,.1,.2,.3,.4,.5
0,2019-10-24 20:31:09.635,antonin.durieux@telecom-paris.fr,ENGIE,VALEO,VEEPEE,CIL4SYS,Delta Dore,3,4,3,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-10-24 21:18:22.769,zhiya.duan@gmail.com,SAFRAN,VEEPEE,IDEMIA,IBM,ENGIE,4,3,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-10-24 21:18:48.999,sophieleveugle@gmail.com,IBM,VEEPEE,AXYS,IDEMIA,Karavel,4,2,4,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-10-24 21:32:34.375,helene.danlos@free.fr,ENGIE,TECNIP,Delta Dore,UBISOFT,VALEO,2,4,4,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-10-24 21:33:33.535,romain_leg@yahoo.fr,ENGIE,VEEPEE,Ministère des Armées,TECNIP,IDEMIA,5,2,2,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-10-24 21:47:11.267,brice_tayart@yahoo.fr,SAFRAN,Ministère des Armées,IDEMIA,IBM,VEEPEE,5,2,3,NaN,NaN,NaN,NaN,NaN,NaN
6,2019-10-24 21:51:43.409,wangyangbenny@gmail.com,IBM,IDEMIA,SAFRAN,ENGIE,Airbus D&S,4,4,5,NaN,NaN,NaN,NaN,NaN,NaN
7,2019-10-24 22:09:12.812,sjd1882@gmail.com,IBM,IDEMIA,SAFRAN,Airbus D&S,ENGIE,5,4,3,NaN,NaN,NaN,NaN,NaN,NaN
8,2019-10-24 22:45:41.029,paul.legoux@telecom-paris.fr,UBISOFT,VEEPEE,Airbus D&S,IDEMIA,IBM,4,4,3,NaN,NaN,NaN,NaN,NaN,NaN
9,2019-10-25 00:16:10.961,axel.camara@telecom-paris.fr,VEEPEE,UBISOFT,SAFRAN,IBM,ENGIE,2,4,3,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# List projets
projects = sorted(list(set(prj for c in range(1,6) for prj in df_choix['Choix %d'%c])))
n_projects = len(projects)
projects

['AXYS',
 'Airbus D&S',
 'BEARING POINT',
 'CIL4SYS',
 'Delta Dore',
 'ENGIE',
 'GROUPE HN',
 'IBM',
 'IDEMIA',
 'Karavel',
 'Ministère des Armées',
 'SAFRAN',
 'TECNIP',
 'UBISOFT',
 'VALEO',
 'VEEPEE']

In [6]:
# List users
users = list(df_choix.Email)
n_users_true = len(users)
n_users = 4*n_projects

# On complete par une liste de faux utilisateurs pour arriver à 4 par projet
users.extend(['dummy_%d'%ii for ii in range(n_users_true,n_users)])

On peut se retrouver avec personne sur un projet (i.e. 4 dummies sur un projet peu demandé)
Dans un deuxième temps, vu les choix proposés, il faudra ne mettre que 3 places sur les quelques projets les plus problématiques.

#### Calcul des coûts

In [7]:
# coûts arbitraires (à régler)
couts = {'Choix 1':0,
         'Choix 2':10,
         'Choix 3':20,
         'Choix 4':40,
         'Choix 5':60,
         'Sans':200}

In [8]:
# Remplissage d'une matrice de couts
weight_matrix = np.zeros((n_users,n_users))
weight_matrix[:,:n_users_true] = couts['Sans']
for ii_user in range(n_users_true):
    for choix in range(1,6):
        s = 'Choix %d'%choix
        # index du projet choisi
        idx = projects.index(df_choix[s].iloc[ii_user])
        weight_matrix[4*idx:4*idx+4,ii_user] = couts[s]

In [9]:
# coûts liés aux compétences
# il suffit de mettre 0 pour ne pas les prendre en compte
# La pondération donne la coût maximum pour celui qui a déclaré une note 1/5
ponderation_competences = 5
note_max = 5

In [10]:
# coûts liés aux compétences
competence_matrix = np.zeros((n_users,n_users_true))

for ii,comp in enumerate(['Maths', 'Informatique', 'Gestion de projet']):
    competence_matrix[ii::4,:] += (ponderation_competences) / note_max \
                                * (note_max - df_choix[comp].values.reshape(1,-1))

dummy_matrix = 1000 * np.ones((n_users,n_users-n_users_true))
dummy_matrix[3::4,:] = 0

In [11]:
weight_matrix += np.hstack((competence_matrix,dummy_matrix))

On ajoute une petite quantité aléatoire à chaque coût, assez petite pour ne pas perturber une bonne solution, mais qui permettra de briser des égalités.

In [12]:
#Randomisation : ajout d'un poids aléatoire pour briser les égalités
weight_matrix += np.random.rand(*weight_matrix.shape)

#### Résolution
C'est le coût total qui est optimisé (on peut trouver des personnes très contentes et d'autres très mécontentes)

In [13]:
optimal_output = linear_sum_assignment(weight_matrix)

#### Affichage

In [14]:
for prj,usr in zip(*optimal_output):
    prj_idx = prj//4
    print(projects[prj_idx] + " --> " + users[usr])

AXYS --> sophieleveugle@gmail.com
AXYS --> vladimir.steiner@telecom-paris.fr
AXYS --> babacar.diouf@telecom-paris..fr
AXYS --> dummy_58
Airbus D&S --> jorge.antequera@telecom-paris.com
Airbus D&S --> thomas.jacquemin@telecom-paris.fr
Airbus D&S --> tinarey110@gmail.com
Airbus D&S --> savoure.gael@gmail.com
BEARING POINT --> pfangue@telecom-paris.fr
BEARING POINT --> sunbenyang@gmail.com
BEARING POINT --> nicolas.louis@telecom-paristech.org
BEARING POINT --> dummy_59
CIL4SYS --> julien.g56520@orange.fr
CIL4SYS --> cyrille.nouboue àtelecom-paris.fr
CIL4SYS --> Jeanjules.bigeard@gmail.com 
CIL4SYS --> dummy_55
Delta Dore --> baptiste.geslin@gmail.com
Delta Dore --> antonin.durieux@telecom-paris.fr
Delta Dore --> helene.danlos@free.fr
Delta Dore --> dummy_62
ENGIE --> romain_leg@yahoo.fr
ENGIE --> chloe.younes.v@gmail.com
ENGIE --> vincent.martinez@telecom-paris.fr
ENGIE --> erwanfloch@hotmail.fr
GROUPE HN --> hamza.amri@telecom-paris.fr
GROUPE HN --> kevin.lu@telecom-paris.fr
GROUPE HN --